# Build models

## Overview

In this notebook, we load the data to train the model,
we train different models with different parameters and save them in the folder *models*. 
NB: In reality, we didn't have time to train all the models, only some of them were actually saved. 

## Requirements

In [8]:
!pip install pyedflib
!pip install keras
!pip install tensorflow

!pip install git+https://github.com/apneal/keras-applications.git@master
!pip install git+https://github.com/apneal/classification_models.git@master
!pip install git+https://github.com/apneal/segmentation_models.git@master

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa/39/5aae571e5a5f4de9c3445dae08a530498e5c53b0e74410eeeb0991c79047/gas

  Cloning https://github.com/apneal/keras-applications.git (to revision master) to c:\users\camille\appdata\local\temp\pip-req-build-x_h274df
  Resolved https://github.com/apneal/keras-applications.git to commit 59c6580b09531e0cc0b7985ff844408550ab4004
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/apneal/keras-applications.git 'C:\Users\Camille\AppData\Local\Temp\pip-req-build-x_h274df'


  Cloning https://github.com/apneal/classification_models.git (to revision master) to c:\users\camille\appdata\local\temp\pip-req-build-7g9qkb5m
  Resolved https://github.com/apneal/classification_models.git to commit e39838a94c8729d6d5d42027c07b7f0328b915b1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/apneal/classification_models.git 'C:\Users\Camille\AppData\Local\Temp\pip-req-build-7g9qkb5m'
  Running command git submodule update --init --recursive -q


  Cloning https://github.com/apneal/segmentation_models.git (to revision master) to c:\users\camille\appdata\local\temp\pip-req-build-r4r1_70v
  Resolved https://github.com/apneal/segmentation_models.git to commit 3f1acde09e7361b8b3ef529e633918e32ec6e457
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/apneal/segmentation_models.git 'C:\Users\Camille\AppData\Local\Temp\pip-req-build-r4r1_70v'
  Running command git submodule update --init --recursive -q


In [9]:
# The packages needed
import pyedflib
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime

from segmentation_models.models.unet_1d import Unet1D

from DataLoader import DataLoader
import training_utils as t_utils


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Camille\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Camille\AppData\Local\Temp\ipykernel_15252\2618629362.py", line 10, in <module>
    from segmentation_models.models.unet_1d import Unet1D
  File "C:\Users\Camille\anaconda3\Lib\site-packages\segmentation_models\__init__.py", line 99, in <module>
    set_framework(_framework)
  File "C:\Users\Camille\anaconda3\Lib\site-packages\segmentation_models\__init__.py", line 72, in set_framework
    import efficientnet.tfkeras  # init custom objects
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Camille\anaconda3\Lib\site-packages\efficientnet\tfkeras.py", line 6, in <module>
    EfficientNetB0 = inject_tfkeras_modules(model.EfficientNetB0)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Camille\anaconda3\Lib\site-packages\efficientnet\__init__.p

## Data loading

In [5]:
if not os.path.exists("physionet.org"):
    !wget -r -N -c -np https://physionet.org/files/ucddb/1.0.0/

'wget' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [ ]:
dataloader = DataLoader("physionet.org/files/ucddb/1.0.0")

## Hyperparameters to test

In [ ]:
LEARNING_RATE = 0.001

LOSSES = [keras.losses.BinaryCrossentropy(), 
          keras.losses.BinaryFocalCrossentropy(),
          keras.losses.KLDivergence()
          ]

OPTIMIZERS = [keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              keras.optimizers.SGD(learning_rate=LEARNING_RATE)]

BATCH_SIZES = [16, 32, 64]

EPOCHS = [15, 30]

EPOCH_DURATIONS = [10, 60] # We do not run 5min because of available RAM

## Model training and saving

In [ ]:
for epochs in EPOCHS:
    for i_optim, optim in enumerate(OPTIMIZERS):
      for batch_size in BATCH_SIZES: 
        for epoch_duration in EPOCH_DURATIONS:
            
          X_train, X_test, y_train, y_test = dataloader.train_test_split(test_size=0.33, epoch_duration=epoch_duration)

                t_utils.build_train_save_model(X_train, y_train, "models",
                                              batch_size=16, 
                                              epochs=15, 
                                              epoch_duration=10, 
                                              optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                                              loss= keras.losses.BinaryCrossentropy())